# Import lib

In [ ]:
# Importing necessary libraries for data manipulation, image processing, and visualization.
import pandas as pd  # Data manipulation and analysis
import numpy as np  # Numerical operations

from PIL import Image  # Image processing using Python Imaging Library (PIL)
import cv2  # Computer vision tasks and image processing with OpenCV
import glob  # File pattern matching for filenames

import matplotlib.pyplot as plt  # Plotting and visualization
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)  # Tick formatting and control in plots
from matplotlib.colors import LightSource  # Light source shading for 3D plots
from matplotlib import cbook, cm  # Utilities for data handling and color mapping
from matplotlib.ticker import PercentFormatter  # Format ticks as percentages

import threading  # For multithreading tasks
from multiprocessing import Process  # For parallel processing using multiple processes


# Main input

In [ ]:
### Input name of the filder with data ###
folder = "2004-11-08"
### Input name of the filder with data ###

# Conversions

In [ ]:
# Function to convert year, month, day, hour, and minute inputs into a standardized datetime format.
# Each input is zero-padded as necessary and combined into a string. 
# Conversion to a NumPy datetime64 object with second precision.
def inp_to_date(year, month, day, hour, minute):
    year, month, day, hour, minute = str(year).zfill(4), str(month).zfill(2), str(day).zfill(2), str(hour).zfill(2), str(minute).zfill(2)
    return np.datetime64(f"{year}-{month}-{day}T{hour}:{minute}", "s")

In [ ]:
# Function to convert year, month, day, hour, and minute inputs into a formatted string for naming dashboards.
# Each input is zero-padded as necessary, then concatenated into a string with the format "YYYYMMDD_HHMM".
def inp_to_img(year, month, day, hour, minute):
    year, month, day, hour, minute = str(year).zfill(4), str(month).zfill(2), str(day).zfill(2), str(hour).zfill(2), str(minute).zfill(2)
    return f"{year}{month}{day}_{hour}{minute}"

In [ ]:
# Function to convert a formatted string (YYYYMMDD_HHMM) which is name of dashboard into a NumPy datetime64 object.
# The string is sliced to extract year, month, day, hour, and minute, then reassembled.
# String is converted into a datetime64 object with second precision.
def img_to_date(date):
    year, month, day, hour, minute = date[0:4], date[4:6], date[6:8], date[9:11], date[11:13]
    return np.datetime64(f"{year}-{month}-{day}T{hour}:{minute}", "s")

In [ ]:
# Function to convert a formatted string (YYYYMMDD_HHMM) into a more readable date-time string used in name of plots.
# The string is sliced to extract year, month, day, hour, and minute, then formatted as "YYYY-MM-DD HH:MM".
def img_to_plot(date):
    year, month, day, hour, minute = date[0:4], date[4:6], date[6:8], date[9:11], date[11:13]
    return f"{year}-{month}-{day} {hour}:{minute}"

In [ ]:
# Function to convert a NumPy datetime64 object into a formatted string (YYYYMMDD_HHMM) for naming dashboards.
# The datetime is first converted to a string, then sliced to extract year, month, day, hour, and minute,
def date_to_img(date):
    temp = np.datetime_as_string(date)
    year, month, day, hour, minute = temp[0:4], temp[5:7], temp[8:10], temp[11:13], temp[14:16]
    return f"{year}{month}{day}_{hour}{minute}"

In [ ]:
# Function to calculate the absolute difference between two datetime64 objects.
# The difference is returned as an integer representing the time difference in seconds.
def dif_time(date1, date2):
    return np.abs((date1 - date2)).astype(int)

# Core generator

In [ ]:
def dash_core(item_date):
    # Convert the datetime64 object to a string and extract year, month, day, hour, and minute.
    item_date = np.datetime_as_string(item_date)
    year, month, day, hour, minute = item_date[0:4], item_date[5:7], item_date[8:10], item_date[11:13], item_date[14:16]

    # Create a datetime object for the input date and define the start and end of the day with a 10-day range.
    date = inp_to_date(year, month, day, hour, minute)
    start_of_day = inp_to_date(year, month, day, 0, 0) - np.timedelta64(10, 'D')
    end_of_day = inp_to_date(year, month, day, 0, 0) + np.timedelta64(10, 'D')

    # Find the nearest dates in various lists and convert them to image paths.
    eit_index = np.argmin(dif_time(date, eit_list))
    LC2_index = np.argmin(dif_time(date, LC2_list))
    mag_index = np.argmin(dif_time(date, mag_list))

    eit_date = date_to_img(eit_list[eit_index])
    LC2_date = date_to_img(LC2_list[LC2_index])
    mag_date = date_to_img(mag_list[mag_index])
    
    N_p_index = np.argmin(dif_time(date, np.array(N_p_list["Time"])))
    V_p_index = np.argmin(dif_time(date, np.array(V_p_list["Time"])))
    B_z_index = np.argmin(dif_time(date, np.array(B_z_list["Time"])))

    # Define file paths for the images.
    eit_path = f"{folder}/soho_eit195/{eit_date}_eit195_1024.png"
    LC2_path = f"{folder}/soho_lasco/c2/{LC2_date}_lascoc2_1024.png"
    mag_path = f"{folder}/soho_mdi/mag/{mag_date}_mdimag_1024.png"

    # Open and process the images for plotting.
    mag_img = Image.open(mag_path).convert('1')
    LC2_img = Image.open(LC2_path)
    eit_img = Image.open(eit_path)

    # Configure and create the plot layout.
    plt.rcParams.update({'font.size': 8})
    fig = plt.figure(figsize=(15, 10), layout='constrained', facecolor="black", edgecolor='white')
    axs = fig.subplot_mosaic([["IMG1", "IMG1","IMG1", "IMG1", "IMG2", "IMG2","IMG2", "IMG2", "IMG3", "IMG3", "IMG3", "IMG3"],
                              ["IMG1", "IMG1","IMG1", "IMG1", "IMG2", "IMG2","IMG2", "IMG2", "IMG3", "IMG3", "IMG3", "IMG3"],
                              ["IMG1", "IMG1","IMG1", "IMG1", "IMG2", "IMG2","IMG2", "IMG2", "IMG3", "IMG3", "IMG3", "IMG3"],
                              ["IMG1", "IMG1","IMG1", "IMG1", "IMG2", "IMG2","IMG2", "IMG2", "IMG3", "IMG3", "IMG3", "IMG3"],
                              ["SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1", "SERIES1"],
                              ["SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2", "SERIES2"],
                              ["SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3", "SERIES3"]])
    
    
    # Display the images in the defined subplot areas.
    axs["IMG1"].imshow(mag_img)
    axs["IMG2"].imshow(eit_img)
    axs["IMG3"].imshow(LC2_img)

    # Customize the appearance of the image subplots.
    for temp_img, temp_name in zip(["IMG1", "IMG2", "IMG3"], [str(img_to_plot(mag_date)), str(img_to_plot(eit_date)), str(img_to_plot(LC2_date))]):
        axs[temp_img].spines['bottom'].set_color('white')
        axs[temp_img].spines['top'].set_color('white')
        axs[temp_img].spines['left'].set_color('white')
        axs[temp_img].spines['right'].set_color('white')
        axs[temp_img].locator_params(axis='y', nbins=10) 
        axs[temp_img].locator_params(axis='x', nbins=10) 
        axs[temp_img].xaxis.set_minor_locator(AutoMinorLocator(10))
        axs[temp_img].yaxis.set_minor_locator(AutoMinorLocator(10))
        axs[temp_img].tick_params(which='major', length=6, direction="inout", colors="white")
        axs[temp_img].tick_params(which='minor', length=2, direction="in", colors="white") 
        axs[temp_img].set_title(temp_name, fontsize = 20)
        axs[temp_img].xaxis.label.set_color('white')
        axs[temp_img].yaxis.label.set_color('white')
        axs[temp_img].tick_params(colors='white', which='both') 
        axs[temp_img].title.set_color('white')

    # Plot time series data on the remaining subplots.
    axs["SERIES1"].plot(np.array(N_p_list.loc[N_p_list["Time"].between(start_of_day, end_of_day)]["Time"]), N_p_list.loc[N_p_list["Time"].between(start_of_day, end_of_day)]["N_p"], color="red", marker=".", linestyle="", markersize=1)
    axs["SERIES2"].plot(np.array(V_p_list.loc[V_p_list["Time"].between(start_of_day, end_of_day)]["Time"]), V_p_list.loc[V_p_list["Time"].between(start_of_day, end_of_day)]["V_p"], color="green", marker=".", linestyle="", markersize=1)
    axs["SERIES3"].plot(np.array(B_z_list.loc[B_z_list["Time"].between(start_of_day, end_of_day)]["Time"]), B_z_list.loc[B_z_list["Time"].between(start_of_day, end_of_day)]["B_z"], color="yellow", marker=".", linestyle="", markersize=1)

    # Customize the appearance of the time series subplots.
    for temp_series, temp_name in zip(["SERIES1", "SERIES2", "SERIES3"], ["rho", "v", "Bz"]):
        axs[temp_series].spines['bottom'].set_color('white')
        axs[temp_series].spines['top'].set_color('white')
        axs[temp_series].spines['left'].set_color('white')
        axs[temp_series].spines['right'].set_color('white')
        axs[temp_series].set_facecolor('black')
        axs[temp_series].locator_params(axis='y', nbins=5) 
        axs[temp_series].yaxis.set_minor_locator(AutoMinorLocator(5))
        axs[temp_series].tick_params(which='major', length=6, direction="inout", colors="white")
        axs[temp_series].tick_params(which='minor', length=2, direction="in", colors="white") 
        axs[temp_series].xaxis.label.set_color('white')
        axs[temp_series].yaxis.label.set_color('white')
        axs[temp_series].tick_params(colors='white', which='both') 
        axs[temp_series].title.set_color('white')

    # Label the y-axes and set shared x-axis labels for the series subplots.
    axs['SERIES1'].set_ylabel(r'Density (cm$^{-3}$)')
    axs['SERIES2'].set_ylabel(r'Speed (cm $\cdot$ s$^{-1}$)')
    axs['SERIES3'].set_ylabel(r'Bz (nT)')
    
    axs["SERIES3"].set_xlabel('Date')
    axs['SERIES2'].sharex(axs['SERIES1'])
    axs['SERIES3'].sharex(axs['SERIES2'])

    # Add vertical lines to indicate the selected time points.
    axs["SERIES1"].axvline(x=N_p_list.iloc[N_p_index]["Time"], color='white', ls='--')
    axs["SERIES2"].axvline(x=N_p_list.iloc[N_p_index]["Time"], color='white', ls='--')
    axs["SERIES3"].axvline(x=N_p_list.iloc[N_p_index]["Time"], color='white', ls='--')

    # Save the plot as a PNG file and close the plot.
    plt.savefig("Dash_" + date_to_img(date) + ".png", dpi=300, facecolor='black', edgecolor='black')
    plt.close()
    # plt.show() # Only if it is neccesary to live view plots

# Main loop

In [ ]:
csv_date = folder[0:4]+folder[5:7]+folder[8:10]

# Generate a list of EIT images' dates by parsing filenames in the specified directory.
eit_list = np.array([temp.split("\\") for temp in glob.glob(f"{folder}/soho_eit195/*")])[::, 1]
eit_list= [img_to_date(temp) for temp in eit_list]

# Generate a list of LC2 images' dates by parsing filenames in the specified directory.
LC2_list = np.array([temp.split("\\") for temp in glob.glob(f"{folder}/soho_lasco/c2/*")])[::, 1]
LC2_list= [img_to_date(temp) for temp in LC2_list]

# Generate a list of magnetic images' dates by parsing filenames in the specified directory.
mag_list = np.array([temp.split("\\") for temp in glob.glob(f"{folder}/soho_mdi/mag/*")])[::, 1]
mag_list= [img_to_date(temp) for temp in mag_list]

# Define paths for CSV files containing in-situ data.
path4 = f"{folder}/in_situ/{csv_date}_soho_N_p.csv"
path5 = f"{folder}/in_situ/{csv_date}_soho_V_p.csv"
path6 = f"{folder}/in_situ/{csv_date}_wind_B_z.csv"            

# Load in-situ data from CSV files and remove unnecessary columns.
N_p_list = pd.read_csv(path4).drop(columns=['Unnamed: 0'])
V_p_list = pd.read_csv(path5).drop(columns=['Unnamed: 0'])
B_z_list = pd.read_csv(path6).drop(columns=['Unnamed: 0'])

# Convert the 'Time' columns in the dataframes to datetime64 format.
N_p_list["Time"] = np.array(N_p_list["Time"]).astype("datetime64")
V_p_list["Time"] = np.array(V_p_list["Time"]).astype("datetime64")
B_z_list["Time"] = np.array(B_z_list["Time"]).astype("datetime64")

# Generate plots for each hour within the range of EIT image dates.
for item_date in np.arange(eit_list[0], eit_list[-1], dtype='datetime64[h]'):
    dash_core(item_date)

# Optionally create a video